# Compute Feature Rankings and Export RRA Scores for Omics Data

This notebook reads interpretation data, computes per-method ranks and RRA scores, and writes the results to an Excel file.


In [ ]:
import torch
import pandas as pd
import numpy as np
from itertools import product

In [ ]:
%%R
install.packages("RobustRankAggreg", repos="https://cloud.r-project.org")
library(RobustRankAggreg)
packageVersion("RobustRankAggreg")
install.packages("writexl")
library(readxl)
library(RobustRankAggreg)

In [ ]:
%%R

# Define method groups
full_methods <- c(
  "cnn_Impact_IxG", "cnn_Impact_IG", "cnn_Impact_GuidedBackprop", "cnn_Impact_Deconvolution",
  "gnn_Impact_IG", "gnn_Impact_Saliency", "gnn_Impact_IxG", "gnn_Impact_Deconvolution",
  "gnn_Impact_GuidedBackprop", "gnn_Impact_GNN_explainer"
)
cnn_methods <- c(
  "cnn_Impact_IxG", "cnn_Impact_IG", "cnn_Impact_GuidedBackprop", "cnn_Impact_Deconvolution"
)
gnn_methods <- c(
  "gnn_Impact_IG", "gnn_Impact_Saliency", "gnn_Impact_IxG", "gnn_Impact_Deconvolution",
  "gnn_Impact_GuidedBackprop", "gnn_Impact_GNN_explainer"
)

# File paths
input_path  <- "data/input_features.xlsx"
output_path <- "results/abs_ranking.xlsx"

In [ ]:
%%R

compute_rankings <- function(data, methods, prefix) {
  for (m in methods) {
    data[[paste0(m, "_rank")]] <- rank(-abs(data[[m]]), ties.method = "min")
  }

  rank_lists <- lapply(methods, function(m) {
    data$feature[order(-abs(data[[m]]))] #ranking by modules
  })
  names(rank_lists) <- methods

  agg <- aggregateRanks(rank_lists)
  agg$agg_rank <- rank(agg$Score, ties.method = "min")

  agg_simple <- data.frame(
    feature           = agg$Name,
    RRA_score_col     = agg$Score,
    RRA_rank_col      = agg$agg_rank,
    stringsAsFactors  = FALSE
  )
  names(agg_simple)[2:3] <- paste0(prefix, c("_RRA_score", "_RRA_rank"))

  merge(data, agg_simple, by = "feature", all.x = TRUE)
}

In [ ]:
%%R

df <- read_excel(input_path)
all_methods <- unique(c(full_methods, cnn_methods, gnn_methods))
df[all_methods] <- lapply(df[all_methods], as.numeric)

df2 <- compute_rankings(df, full_methods, "full")
df2 <- compute_rankings(df2, cnn_methods,  "cnn")
df2 <- compute_rankings(df2, gnn_methods,  "gnn")

write_xlsx(df2, path = output_path)